In [ ]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
import numpy as np
from IPython.display import display, Markdown

Copyright **Jacob Martin and Paolo Raiteri**, January 2021

## Statistical Analysis and Linear Regression
In the short introductory lecture to this workshop we have learnt how to perform linear regression both manually and using Excel. We can now look at a range of Physics/Chemistry problems where we can use linear regression to fit apparently non linear equations.

1. *Random numbers <p></p>

2. *Lambert-Beer Law*: $A = \varepsilon l c$ <p></p>

3. *Aerodynamic drag force*: $F_D = \dfrac{1}{2}\rho C_D A v^2$ <p></p>

4. *Electrostatic force*: $F_E = \dfrac{1}{4\pi\epsilon_0}\dfrac{q^2}{\varepsilon_r r^2}$ <p></p>

5. *Arrhenius*: $K_{r} = Ae^{-\Delta E_a/RT}$ <p></p>

6. *van't Hoff equation*: $K_{eq} = e^{-\Delta G^0/RT}$ <p></p>

In this and the following workshops we will exclusively use Excel to numerically solve the exercise.
At the beginning we will solve problems using simple linear regression, but later we will learn how to tackle multivariable problems using a self consistent approach.

There are of course other tools that are more sofisticated that we could be using, such as R and Python, but in this instance we decided to stick with the tool most of you are familiar with. However, anyone who is interested in using/learning R and/or Python is encouraged to do so, and we can provide some guidance about how to solve these problems using those tools.
In fact, all this workshops have been written in Python using Jupyter Notebooks.

### General instructions:

- Select one of the `tasks` that are available from the dropdown menu below
- Select the number of data points you want to collect
- Click `Generate Data` to obtain the data set
- Click `Download CSV` to export the complete data set for all the experiments as a CSV file
- Alternatively you can copy the data and later paste them into your spreadsheet

###  Notes and Tips:
- The data set is designed to include error and uncertainties of the experiments, so they contain a (variable) amount of noise
- The data set is produced with a fixed number of digits, but like in experiments, not all of them may be significative.
- You can initially generate a large data set and use smaller subset to assess the convegency of the result
- Every time you click `Generate Data` an entirely new data set is produced

### Discussion points for all task:
- For each task, discuss with your peers and/or the lab demonstrations your impression about the convergence of the computed quantity with the number of data points.
- How many points do you need to produce a reliable estimate?

### Specific aims for each task:
1. *Random numbers*:
    - Compute the average, standard deviation, and standard error using different number of points. 
    - Compute the histogram of the frequency of the observations and discuss the relevance of the standard deviation in the two cases.
    - Extra #1, plot of the running average, _i.e_ the average as a function of the number of observations
    - Extra #2, plot of the standard deviation as a function of the number of observations
    - Extra #3, plot of the standard error as a function of the number of observations
    - Extra #4, plot the normalised histrogram and the "normal" distribution on the same graph <p></p>


2. *Lambert-Beer law*:
    - Use an excel spreadsheet to compute the absorptivity, $\varepsilon$, assuming that the measurements were performed in a cuvette with an optical path length of 0.5 cm <p></p>

3. *Aerodynamic drag force*:
    - Determine the aerodynamic drag coefficient ($C_x = \rho C_D A$)
    - Try to use a polynomial "trend line" and compare the quality of the fit
    - Is there a problem with the polynomial fit (_at least in Excel_)?  <p></p>

4. *Electrostatic force*:
    - Determine the dielectric constant of the medium ($\varepsilon_r$) assuming $q^2=1$
    - _Tip: find the value of the Coulomb constant in eV_ <p></p>

5. *Arrhenius*:
    - Determine the activation energy ($E_a$)
    - Determine the frequency factor ($A$)
    - Try to use an exponential "trend line" and compare the result with the linear fit
    <p></p>
    

6. *van't Hoff equation*:
    - Determine the standard enthalpy and entropy of the reaction; remember that $\Delta G = \Delta H - T\Delta S$
    - What is the equilibrium constant at 32 degrees Celsius?



In [ ]:
# define path to results.csv file
respath = os.path.join(os.getcwd(), "results.csv")

# delete existing result file and setup rng
if os.path.exists(respath):
    os.remove(respath)

#random.seed(params["error"].get("seed", 0))
t = int( time.time() * 1000.0 )
iseed = (((t & 0xff000000) >> 24) + 
         ((t & 0x00ff0000) >>  8) +
         ((t & 0x0000ff00) <<  8) +
         ((t & 0x000000ff) << 24)  )

random.seed(iseed)
np.random.seed(iseed)

def selectExercise(x, l1, l2):
    global xlabel, ylabel
    global exercise
    exercise = x
    xlabel = l1
    ylabel = l2

In [ ]:
def func(iType,x):
    ymin = 6
    ymax = 20
    eps = 0.006
    m = 7.4 
    e = 65.
    k = 0.2
    dH = 88500
    dS = 281
    ea = 1
    
    if (iType == 0):
        res = (ymax-ymin)*random.random() + ymin
        
    elif (iType == 1):
        mu, sigma = ymax, 2 # mean and standard deviation
        res = float(np.random.normal(mu, sigma, 1))
        
    elif (iType == 2):
        res = x * eps * 0.5
        
    elif (iType == 3):
        res =  0.5 * m * x*x
        
    elif (iType == 4):
        res = 143.9 / e / x/x

    elif (iType == 5):
        kt = 8.3145 * x
        dg = dH - x*dS
        res = math.exp(-dg/kt)

    elif (iType == 6):
        kt = 8.617e-5 * x
        a = 2.1e12
        res = a * math.exp(-ea/kt)      
        
    return res
    
    
def measure1(iType,nPoints):
    dataSet = pd.DataFrame()
    scatter = 0.3

    xmin = 0
    xmax = 10

    if (iType == 0) or (iType == 1):
        scatter = 0
        
    if (iType == 1):
        xmin = 0 
        xmax = 1
    elif (iType == 4):
        xmin = 0.1
        xmax = 1
    elif (iType == 5) or (iType == 6):
        xmin = 273
        xmax = 473
        scatter = 1

    for i in range(0,nPoints):
        m = 3.7
        if (iType == 1) or (iType == 0):
            xval = i
        else:
            xval = (xmax-xmin)*random.random() + xmin
        yval = func(iType,xval) 
        yval = yval + scatter * yval * (random.random()-0.5)
        data = [{"x" : xval,"y" : yval}]
        dataSet = dataSet.append(pd.DataFrame(data))
        
        
    return dataSet

In [ ]:
out_P = ipw.Output()
out_L = ipw.Output()

#setNumberOfDataPoints(10)
selectExercise(0, "X", "Y")

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))

    
def calc1(btn):
    out_P.clear_output()
    
    if os.path.exists(respath):
        os.remove(respath)
    res = pd.DataFrame(columns=[xlabel , ylabel])

    # Perform N measurements
    nn = int(textID.value)
    data = measure1(exercise,nn)
    
    for i in range(0, nn):
        var_list = []
        var_list.append(data.iloc[i,0])
        var_list.append(data.iloc[i,1])
    
        # Append result
        res.loc[len(res)] = var_list
        
    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(nn))

btn_submit = ipw.Button(description="Submit", layout=ipw.Layout(width="150px"))

btn_calc1 = ipw.Button(description="Get Data", layout=ipw.Layout(width="150px"))
btn_calc1.on_click(calc1)

rows = []

# Equation
w = ipw.Dropdown(
    options=["Random #1", 
             "Random #2", 
             "Lambert-Beer Law", 
             "Aerodynamic drag Force", 
             "Electrostatic force",
             "Equilibrium constant",
             "Arrhenius"
            ],
    value='Random #1',
    description='Task',
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if (change['new'] == "Random #1"):
            selectExercise(0, "N", "Y")
        elif (change['new'] == "Random #2"):
            selectExercise(1, "N", "Y")
        elif (change['new'] == "Lambert-Beer Law"):
            selectExercise(2, "$c$ [mol/L]", "$A$")
        elif (change['new'] == "Aerodynamic drag Force"):
            selectExercise(3, "$v$ [m/s]", "$F_D$ [N]")
        elif (change['new'] == "Electrostatic force"):
            selectExercise(4, "$r$ [nm]", "$F_E$ [eV/nm]")
        elif (change['new'] == "Equilibrium"):
            selectExercise(5, "$T$ [K]", "$K_{eq}$")
        elif (change['new'] == "Arrhenius"):
            selectExercise(6, "$T$ [K]", "$K_r$ [mol/L/s]")

w.observe(on_change)

rows.append(ipw.HBox([w]))

# Nmber of data points
textID = ipw.Text("10")

rows.append(ipw.HBox([ipw.Label('Number of data point required:  ')]))
rows.append(ipw.HBox([textID]))
rows.append(ipw.HBox([btn_calc1]))

rows.append(ipw.HBox([out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)
